<a href="https://colab.research.google.com/github/sripraks/CNN/blob/main/flowers/finetune_flowers17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Train VGG 16 on Flower Data set and Fine Tune With Augmentation + Top Layers**

In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import img_to_array

class ImageToArrayPreprocessor:
	def __init__(self, dataFormat=None):
		# store the image data format
		self.dataFormat = dataFormat

	def preprocess(self, image):
		# apply the Keras utility function that correctly rearranges
		# the dimensions of the image
		return img_to_array(image, data_format=self.dataFormat)

In [ ]:
# import the necessary packages
import imutils
import cv2

class AspectAwarePreprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# store the target image width, height, and interpolation
		# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter

	def preprocess(self, image):
		# grab the dimensions of the image and then initialize
		# the deltas to use when cropping
		(h, w) = image.shape[:2]
		dW = 0
		dH = 0

		# if the width is smaller than the height, then resize
		# along the width (i.e., the smaller dimension) and then
		# update the deltas to crop the height to the desired
		# dimension
		if w < h:
			image = imutils.resize(image, width=self.width,
				inter=self.inter)
			dH = int((image.shape[0] - self.height) / 2.0)

		# otherwise, the height is smaller than the width so
		# resize along the height and then update the deltas
		# crop along the width
		else:
			image = imutils.resize(image, height=self.height,
				inter=self.inter)
			dW = int((image.shape[1] - self.width) / 2.0)

		# now that our images have been resized, we need to
		# re-grab the width and height, followed by performing
		# the crop
		(h, w) = image.shape[:2]
		image = image[dH:h - dH, dW:w - dW]

		# finally, resize the image to the provided spatial
		# dimensions to ensure our output image is always a fixed
		# size
		return cv2.resize(image, (self.width, self.height),
			interpolation=self.inter)

In [ ]:
# import the necessary packages
import numpy as np
import cv2
import os

class SimpleDatasetLoader:
	def __init__(self, preprocessors=None):
		# store the image preprocessor
		self.preprocessors = preprocessors

		# if the preprocessors are None, initialize them as an
		# empty list
		if self.preprocessors is None:
			self.preprocessors = []

	def load(self, imagePaths, verbose=-1):
		# initialize the list of features and labels
		data = []
		labels = []

		# loop over the input images
		for (i, imagePath) in enumerate(imagePaths):
			# load the image and extract the class label assuming
			# that our path has the following format:
			# /path/to/dataset/{class}/{image}.jpg
			image = cv2.imread(imagePath)
			label = imagePath.split(os.path.sep)[-2]

			# check to see if our preprocessors are not None
			if self.preprocessors is not None:
				# loop over the preprocessors and apply each to
				# the image
				for p in self.preprocessors:
					image = p.preprocess(image)

			# treat our processed image as a "feature vector"
			# by updating the data list followed by the labels
			data.append(image)
			labels.append(label)

			# show an update every `verbose` images
			if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
				print("[INFO] processed {}/{}".format(i + 1,
					len(imagePaths)))

		# return a tuple of the data and labels
		return (np.array(data), np.array(labels))

In [ ]:
# import the necessary packages
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class FCHeadNet:
	@staticmethod
	def build(baseModel, classes, D):
		# initialize the head model that will be placed on top of
		# the base, then add a FC layer
		headModel = baseModel.output
		headModel = Flatten(name="flatten")(headModel)
		headModel = Dense(D, activation="relu")(headModel)
		headModel = Dropout(0.5)(headModel)

		# add a softmax layer
		headModel = Dense(classes, activation="softmax")(headModel)

		# return the model
		return headModel

In [ ]:
# import the necessary packages
from tensorflow.keras.applications import VGG16

In [ ]:
# load the VGG16 network
print("[INFO] loading network...")
model = VGG16(weights="imagenet",include_top=True)
print("[INFO] showing layers...")

[INFO] loading network...
553467904/553467096 [==============================] - 3s 0us/step
[INFO] showing layers...


In [ ]:
# loop over the layers in the network and display them to the
# console
for (i, layer) in enumerate(model.layers):
	print("[INFO] {}\t{}".format(i, layer.__class__.__name__))

[INFO] 0	InputLayer
[INFO] 1	Conv2D
[INFO] 2	Conv2D
[INFO] 3	MaxPooling2D
[INFO] 4	Conv2D
[INFO] 5	Conv2D
[INFO] 6	MaxPooling2D
[INFO] 7	Conv2D
[INFO] 8	Conv2D
[INFO] 9	Conv2D
[INFO] 10	MaxPooling2D
[INFO] 11	Conv2D
[INFO] 12	Conv2D
[INFO] 13	Conv2D
[INFO] 14	MaxPooling2D
[INFO] 15	Conv2D
[INFO] 16	Conv2D
[INFO] 17	Conv2D
[INFO] 18	MaxPooling2D
[INFO] 19	Flatten
[INFO] 20	Dense
[INFO] 21	Dense
[INFO] 22	Dense


In [ ]:
# load the VGG16 network
print("[INFO] loading network...")
model = VGG16(weights="imagenet",include_top=False)
print("[INFO] showing layers...")

[INFO] loading network...
58892288/58889256 [==============================] - 1s 0us/step
[INFO] showing layers...


In [ ]:
# loop over the layers in the network and display them to the
# console
for (i, layer) in enumerate(model.layers):
	print("[INFO] {}\t{}".format(i, layer.__class__.__name__))

[INFO] 0	InputLayer
[INFO] 1	Conv2D
[INFO] 2	Conv2D
[INFO] 3	MaxPooling2D
[INFO] 4	Conv2D
[INFO] 5	Conv2D
[INFO] 6	MaxPooling2D
[INFO] 7	Conv2D
[INFO] 8	Conv2D
[INFO] 9	Conv2D
[INFO] 10	MaxPooling2D
[INFO] 11	Conv2D
[INFO] 12	Conv2D
[INFO] 13	Conv2D
[INFO] 14	MaxPooling2D
[INFO] 15	Conv2D
[INFO] 16	Conv2D
[INFO] 17	Conv2D
[INFO] 18	MaxPooling2D


In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from imutils import paths
import numpy as np
import argparse
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [ ]:
# grab the list of images that we'll be describing, then extract
# the class label names from the image paths
print("[INFO] loading images...")
imagePaths = list(paths.list_images("/content/drive/My Drive/dataset/flowers17/images"))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

[INFO] loading images...


In [ ]:
# initialize the image preprocessors
aap = AspectAwarePreprocessor(224, 224)
iap = ImageToArrayPreprocessor()

In [ ]:
# load the dataset from disk then scale the raw pixel intensities to
# the range [0, 1]
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0

[INFO] processed 500/1360
[INFO] processed 1000/1360


In [ ]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)

In [ ]:
# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [ ]:
# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
# initialize the new head of the network, a set of FC layers
# followed by a softmax classifier
headModel = FCHeadNet.build(baseModel, len(classNames), 256)

In [ ]:
# place the head FC model on top of the base model -- this will
# become the actual model we will train
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
# loop over all layers in the base model and freeze them so they
# will *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")
opt = RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [ ]:
# train the head of the network for a few epochs (all other
# layers are frozen) -- this will allow the new FC layers to
# start to become initialized with actual "learned" values
# versus pure random
print("[INFO] training head...")
model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
	validation_data=(testX, testY), epochs=25,
	steps_per_epoch=len(trainX) // 32, verbose=1)

[INFO] training head...
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
31/31 [==============================] - 14s 459ms/step - loss: 4.7478 - accuracy: 0.1609 - val_loss: 2.6238 - val_accuracy: 0.1912
Epoch 2/25
31/31 [==============================] - 12s 388ms/step - loss: 2.1445 - accuracy: 0.3259 - val_loss: 1.7469 - val_accuracy: 0.4324
Epoch 3/25
31/31 [==============================] - 12s 391ms/step - loss: 1.9032 - accuracy: 0.3877 - val_loss: 1.4760 - val_accuracy: 0.5441
Epoch 4/25
31/31 [==============================] - 12s 387ms/step - loss: 1.6475 - accuracy: 0.4696 - val_loss: 1.0756 - val_accuracy: 0.6971
Epoch 5/25
31/31 [==============================] - 12s 388ms/step - loss: 1.3954 - accuracy: 0.5597 - val_loss: 0.9785 - val_accuracy: 0.7118
Epoch 6/25
31/31 [==============================] - 12s 390ms/step - loss: 1.4319 - accuracy: 0.5425 - val_loss: 0.9635 - val_accuracy: 0.6853
Epoch 7/25
31/31 [========================

In [ ]:
# evaluate the network after initialization
print("[INFO] evaluating after initialization...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=classNames))

[INFO] evaluating after initialization...
              precision    recall  f1-score   support

    bluebell       0.74      0.77      0.76        22
   buttercup       0.93      0.78      0.85        18
   coltsfoot       0.55      1.00      0.71        16
     cowslip       0.75      0.52      0.62        23
      crocus       0.85      0.96      0.90        23
    daffodil       0.72      0.65      0.68        20
       daisy       1.00      1.00      1.00        18
   dandelion       0.85      0.73      0.79        15
  fritillary       0.94      1.00      0.97        16
        iris       0.94      0.85      0.89        20
  lilyvalley       0.90      0.95      0.92        19
       pansy       0.94      0.85      0.89        20
    snowdrop       0.95      0.74      0.83        27
   sunflower       0.82      1.00      0.90        23
   tigerlily       0.95      1.00      0.98        21
       tulip       0.64      0.47      0.55        19
  windflower       0.79      0.95      

In [ ]:
# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in baseModel.layers[15:]:
	layer.trainable = True

In [ ]:
# for the changes to the model to take affect we need to recompile
# the model, this time using SGD with a *very* small learning rate
print("[INFO] re-compiling model...")
opt = SGD(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] re-compiling model...


In [ ]:
# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
print("[INFO] fine-tuning model...")
model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
	validation_data=(testX, testY), epochs=100,
	steps_per_epoch=len(trainX) // 32, verbose=1)

[INFO] fine-tuning model...
Epoch 1/100
31/31 [==============================] - 13s 409ms/step - loss: 0.4754 - accuracy: 0.8472 - val_loss: 0.3880 - val_accuracy: 0.8824
Epoch 2/100
31/31 [==============================] - 13s 404ms/step - loss: 0.4293 - accuracy: 0.8543 - val_loss: 0.3446 - val_accuracy: 0.8882
Epoch 3/100
31/31 [==============================] - 13s 407ms/step - loss: 0.3846 - accuracy: 0.8694 - val_loss: 0.4090 - val_accuracy: 0.8824
Epoch 4/100
31/31 [==============================] - 13s 410ms/step - loss: 0.3634 - accuracy: 0.8725 - val_loss: 0.3562 - val_accuracy: 0.8941
Epoch 5/100
31/31 [==============================] - 12s 402ms/step - loss: 0.3517 - accuracy: 0.8826 - val_loss: 0.3632 - val_accuracy: 0.8971
Epoch 6/100
31/31 [==============================] - 12s 401ms/step - loss: 0.3574 - accuracy: 0.8745 - val_loss: 0.3267 - val_accuracy: 0.9029
Epoch 7/100
31/31 [==============================] - 12s 402ms/step - loss: 0.3639 - accuracy: 0.8765 - val_

In [ ]:
# evaluate the network on the fine-tuned model
print("[INFO] evaluating after fine-tuning...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=classNames))

[INFO] evaluating after fine-tuning...
              precision    recall  f1-score   support

    bluebell       0.86      0.86      0.86        22
   buttercup       0.95      1.00      0.97        18
   coltsfoot       0.94      1.00      0.97        16
     cowslip       0.91      0.87      0.89        23
      crocus       1.00      0.96      0.98        23
    daffodil       0.83      1.00      0.91        20
       daisy       1.00      1.00      1.00        18
   dandelion       1.00      0.93      0.97        15
  fritillary       0.94      1.00      0.97        16
        iris       0.94      0.85      0.89        20
  lilyvalley       0.90      0.95      0.92        19
       pansy       0.94      0.85      0.89        20
    snowdrop       0.93      0.93      0.93        27
   sunflower       1.00      0.91      0.95        23
   tigerlily       1.00      0.95      0.98        21
       tulip       0.79      0.79      0.79        19
  windflower       0.86      0.95      0.9